# 2. Extraction des données (from XML to CSV by LXML)

Ce notebook extrait les paragraphes des CRS de l’Assemblée nationale à partir des fichiers XML, en utilisant la bibliothèque lxml (qui a ici donné des résultats plus simples que la précédente version). Sans doute moins robuste, moins de gestion des erreurs et exceptions, mais plus simple, lisible, et sans doute OK sur les données assemblée qui sont stables / propres.

Sur le choix de cette base de données plus qu'une autre voir le notebook 1. Data collection

### Pré-Étape :
Supprimer manuellement le premier fichier xml du dossier de la 16e législature qui correspond en réalité à la 15e législature !
ID : "CRSANR5L16S2021O1N144"

## Modalités de fonctionnement : 

### Option 1 : 

1. Fusion des xml par législature
2. Fusionner les 2 CSV (avant ou après le match avec données sur les députés)

### Option 2 : 

1. Fusion de tous les xml dans un seul fichier CSV

1 député = 1 ID qui ne change pas à travers le temps. ==> ce qui signifie que dans l’idéal il faudrait avoir les données complètes (avec affiliation partisane, etc) par législature et faire la fusion député/compte-rendus avant la fusion entre les 2 législatures. 

## Étape 1 : Fusionner les xml par législature dans un CSV

**Faire attention à ne pas perdre de données en routes, et garder en tête l'architecture de construction des xml (cf 1. Data collection)**

In [ ]:
import os #bibliothèque pour traiter les données sur l'ordi
import glob
from lxml import etree #bibliothèque pour traiter les données XML 
import pandas as pd #bibliothèque pour traiter les données tabulaires


def extraire_paragraphes_lxml(fichier_xml: str) -> pd.DataFrame: # Étape 1 consiste à extraire les paragraphes d'un fichier XML de compte rendu en utilisant lxml
    try:
        tree = etree.parse(fichier_xml) #analyser la structure du xml
        root = tree.getroot() # root est la racine de l’arbre XML, c’est-à-dire l’élément XML principal.
        ns = {'ns': 'http://schemas.assemblee-nationale.fr/referentiel'} # ns est un dictionnaire des espaces de noms XML. Le préfixe 'ns' est utilisé pour faire des recherches XPath dans le bon namespace (et ici celui de l'AN donné)

        meta = {
            'UID': root.findtext('ns:uid', namespaces=ns),
            'SeanceRef': root.findtext('ns:seanceRef', namespaces=ns), # donnée manquante pour les xml de la 15e législature
            'SessionRef': root.findtext('ns:sessionRef', namespaces=ns), # pareil
        } # ici il s'agit d'extraire les métadonnées essentielles (la première étant la seule généralisée)
        meta_tags = ['dateSeance', 'dateSeanceJour', 'numSeanceJour', 'numSeance', 'typeAssemblee', 
                     'legislature', 'session', 'nomFichierJo'] #ici on liste les balises complémentaires
        for tag in meta_tags:
            meta[tag] = root.findtext(f'.//ns:{tag}', namespaces=ns) #cela signifie que pour chaque balise, on extrait le texte grâce à la fonction .findtext

        meta['President'] = root.findtext('.//ns:presidentSeance', namespaces=ns)

        rows = []

        for para in root.xpath('.//ns:paragraphe', namespaces=ns): #On récupère tous les éléments <paragraphe> dans le document XML via XPath
            # Naviguer vers le <point> parent (représente généralement l'unité logique du débat)
            point = para.getparent()
            while point is not None and point.tag != f"{{{ns['ns']}}}point":
                point = point.getparent()
            # Extraction des attributs du paragraphe (<point>)
            point_id = point.get('id_syceron') if point is not None else None # simple arbre à conditions pour mettre données manquantes le cas échéant
            point_type = point.get('code_grammaire') if point is not None else None
            point_title = point.findtext('ns:texte', namespaces=ns) if point is not None else None
            valeur_odj = point.get('valeur_ptsodj') if point is not None else None
            # Extraction du texte de l'intervention
            texte_elem = para.find('ns:texte', namespaces=ns)
            texte = ''.join(texte_elem.itertext()).strip() if texte_elem is not None else None
            stime = texte_elem.get('stime') if texte_elem is not None else None  #absent pour la 15e législature
            # Extraction des informations sur l'orateur du paragraphe 
            orateur = para.find('.//ns:orateur', namespaces=ns)
            nom = orateur.findtext('ns:nom', namespaces=ns) if orateur is not None else None
            qualite = orateur.findtext('ns:qualite', namespaces=ns) if orateur is not None else None
            id_orateur = orateur.findtext('ns:id', namespaces=ns) if orateur is not None else None

            rows.append({
                'UID': meta['UID'], # le nom de chaque fichier/CRS
                'SeanceRef': meta['SeanceRef'],
                'SessionRef': meta['SessionRef'],
                'dateSeance': meta['dateSeance'],
                'dateSeanceJour': meta['dateSeanceJour'], # en supprimer un des 2 ?
                # 'numSeanceJour': meta['numSeanceJour'], pour cette étude pas nécessaire, élément intéressant dans des cas spécifiques à aller voir au cas par cas mais ici rajoute colonne inutilement
                # 'numSeance': meta['numSeance'], pareil
                'typeAssemblee': meta['typeAssemblee'],
                'legislature': meta['legislature'],
                'session': meta['session'],
                'nomFichierJO': meta['nomFichierJo'],
                'President': meta['President'],
                'titre_general': point_title,
                # 'Sous_titre': '',  # not in this version, get back to original if needed
                # 'Contexte_hierarchique': '',  # not in this version, get back to original if needed
                # 'Section_courante': '',  # not in this version, get back to original if needed
                # 'Sujet_point': '', # not in this version, get back to original if needed
                'valeur_ODJ': valeur_odj, # potentiellement intéressant pour vérifier si une forte occurence provient d'un même point ou de deux points différents à l'ODJ lors de la même séance
                'point_ID': point_id,
                'point_type': point_type,
                'ID_paragraphe': para.get('id_syceron'), # élement clé pour l'analyse de discours ensuite 
                'ordre_seance': para.get('ordre_absolu_seance'),
                'code_grammaire': para.get('code_grammaire'),
                'code_style': para.get('code_style'),
                'code_parole': para.get('code_parole'),
                'role_debat': para.get('roledebat'), # en réalité trop lacunaire donc pourrait être supprimé
                'nom_orateur': nom,
                'qualite_orateur': qualite, # utilisé essentiellement pour les ministres ou les rapporteur.ices
                'ID_orateur': id_orateur, # élément clé pour commencer à sociologiser l'analyse de discours ensuite 
                'stime': stime, # absent des XML de la 15e législature (correspond au temps de parole)
                'texte': texte # coeur de l'analyse de discours 
            })

        return pd.DataFrame(rows) # renvoie tout ça sous la forme d'un tableau avec les données en lignes. 

    except Exception as e:
        print(f"Erreur dans {fichier_xml} : {e}")
        return pd.DataFrame()


def dossier_CR_lxml(dossier_path: str, pattern: str = "*.xml") -> pd.DataFrame: #Traite tous les fichiers XML d'un dossier avec la fonction extraire_paragraphes_lxml().
    fichiers = glob.glob(os.path.join(dossier_path, pattern))
    if not fichiers:
        print(f"Aucun fichier XML trouvé dans {dossier_path}")
        return pd.DataFrame()

    all_dfs = []
    total = len(fichiers)
    print(f"Traitement de {total} fichiers XML...\n")

    for i, fichier in enumerate(fichiers, 1):
        nom = os.path.basename(fichier)
        print(f"[{i}/{total}] {nom}...", end=" ")

        df = extraire_paragraphes_lxml(fichier)
        if not df.empty:
            print(f" {len(df)} lignes")
            all_dfs.append(df)
        else:
            print("Vide ou erreur")

    if all_dfs:
        df_final = pd.concat(all_dfs, ignore_index=True)
        print(f"\n Export terminé : {len(df_final)} lignes consolidées")
        return df_final
    else:
        return pd.DataFrame()



In [5]:
df = dossier_CR_lxml("/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Raw/16e/")
df.to_csv("/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_16e.csv", index=False, encoding='utf-8')
print(f"\n Export CSV : ({df.shape[0]} lignes)")

# Si volonté de faire directement un CSV utiliser le dossier où j'ai enregistré tous les xml indépendamment de leur législature. 
# Sinon, utiliser les dossiers de sortie du téléchargement sur le site de l'AN. 

Traitement de 604 fichiers XML...

[1/604] CRSANR5L16S2023O1N173.xml...  633 lignes
[2/604] CRSANR5L16S2023O1N167.xml...  467 lignes
[3/604] CRSANR5L16S2023O1N198.xml...  402 lignes
[4/604] CRSANR5L16S2023E1N010.xml...  818 lignes
[5/604] CRSANR5L16S2024O1N117.xml...  678 lignes
[6/604] CRSANR5L16S2024O1N103.xml...  857 lignes
[7/604] CRSANR5L16S2023E1N004.xml...  646 lignes
[8/604] CRSANR5L16S2023O1N239.xml...  452 lignes
[9/604] CRSANR5L16S2023O1N205.xml...  118 lignes
[10/604] CRSANR5L16S2023O1N211.xml...  296 lignes
[11/604] CRSANR5L16S2024O1N088.xml...  403 lignes
[12/604] CRSANR5L16S2023O1N007.xml...  535 lignes
[13/604] CRSANR5L16S2023O1N013.xml...  808 lignes
[14/604] CRSANR5L16S2024O1N063.xml...  106 lignes
[15/604] CRSANR5L16S2024O1N077.xml...  760 lignes
[16/604] CRSANR5L16S2024O1N076.xml...  834 lignes
[17/604] CRSANR5L16S2024O1N062.xml...  322 lignes
[18/604] CRSANR5L16S2023O1N012.xml...  688 lignes
[19/604] CRSANR5L16S2023O1N006.xml...  430 lignes
[20/604] CRSANR5L16S2024

## Étape 2 : Fusionner données députés pour chaque législature 

1 député = 1 ID qui ne change pas à travers le temps. ==> ce qui signifie que dans l’idéal il faudrait avoir les données complètes (avec affiliation partisane, etc) par législature et faire la fusion député/compte-rendus avant la fusion entre les 2 législatures. 

En attendant, possible de lancer les analyses et de recouper en 2 le fichier pour ajouter les députés avant de fusionner à nouveau ensuite

### Quelles options possibles et disponibles ? 


#### L'option de base lacunaire : les informations d'état civique donné par l'AN 

Jeu de données en open acces disponible sur le site open data de l'Assemblée nationale avec État civil des personnes qui sont ou qui ont été députés, sénateurs et ministres (précisant pour ces derniers l'intitulé de leur ministère)
--> **Gros soucis puisque ne contient par leur affiliation politique**
- https://data.assemblee-nationale.fr/archives-anterieures/archives-15e/deputes-senateurs-et-ministres
- https://data.assemblee-nationale.fr/archives-16e/deputes-senateurs-ministres
--> dispo en json ou xml

En théorie, il serait possible de retravailler nous-même ce fichier en le combinant avec d'autres pour avoir leur affiliation politique. En pratique, a déjà été réalisé par d'autres. 


#### L'option la plus facile/accessible et moins couteuse dans un premier temps : la base de données de DATAN 

Ce travail de fusion des données ouvertes de l'AN a été réalisé par Awenig Marié (post-doc au social media lab au CEVIPOl de l'ULB à Bruxelle, contact awenig.marie@ulb.be) via le site/outil DATAN (initiative aux frontières de l'académique avec un conseil scientifique présidé par Olivier Costa, et présence notamment de Isabelle Guinaudeau et Olivier Rozenberg). Les contacter via info[at]datan.fr

Ce site/outil DATAN propose en open data (sur leur site et sur data.gouv) des informations et statistiques sur les groupes parlementaires et les députés. Il met notamment à disposition un jeu de données en .CSV qui présente tous les députés de l'Assemblée nationale française depuis la 12ème législature (2002) avec notamment leur historique parlementaire et leur actuel groupe politique.

Les variables sont les suivantes :

- id : id présente sur le site de l'Assemblée
- legislatureLast : dernière législature où le député a eu un mandat
- civ : genre du député
- nom
- prenom
- villeNaissance : ville de naissance
- naissance : date de naissance
- age
- groupe : actuel groupe parlementaire
- groupeAbrev : abréviation du groupe
- departementNom : nom du département dans lequel le député a été élu
- departementCode : code du département dans lequel le député a été élu
- circo : numéro de sa circonscription
- datePriseFonction : date à laquelle il a pris fonction lors de la législature actuelle
- job
- mail
- twitter
- facebook
- website
- nombreMandats: le nombre de mandats parlementaires exercé par le député
- experienceDepute : expérience parlementaire (en jours, mois ou années)
- scoreParticipation : le score participation correspond au pourcentage de scrutins auquel le député a pris part en séance publique
- scoreParticipationSpectialite : le score de participation "spécialité" correspond au pourcentage de scrutins auquel le député a pris part en séance publique et qui sont reliés à un texte législatif examiné dans la commission parlementaire du député, et donc pour lequel le député a un intérêt ou une expertise
- scoreLoyaute : le score de loyauté correspond au pourcentage de scrutins pour lequel le député a voté sur la même ligne que son groupe parlementaire
- scoreMajorite : le score de proximité avec la majorité présidentielle correspond au pourcentage de scrutins pour lequel le député a voté sur la même ligne que le groupe de la majorité présidentielle
- active : "1" si le député est encore actif
- dateMaj : dernière date de mise à jour du jeu de données

==> Awenig Marié a été contacté et devrait prochainement transmettre un fichier CSV par législature étudiée (15e et 16e) avec les variables suivantes 

À compléter une fois données récupérées 

https://datan.fr/deputes/legislature-15
https://datan.fr/deputes/legislature-16


#### Les critiques faites aux données de l'AN 

Critiques récurrentes dans les travaux de Boelaert, Michon et Ollion, sur l’usage habituel des données de l’AN pour étudier les députés. Il s’agit de **données auto-déclaratives sujettes à la mal-déclaration, aux stratégies de présentations de soi** (mettre en avant une profession ou expérience que l’on juge plus valorisante), et à une inadéquation des catégories et données à remplir (: exemple de la profession d’origine qui ne prend pas en compte le temps passé en politique depuis).


#### Les alternatives à considérer pour plus tard 

1. Demander le jeu de données constitué par l'équipe autour de Ollion, Michon et Boelaert 
    - Systématisation d'une collecte documentaire visant à noter chaque profession occupée par les députés à partir de leurs 25 ans via les articles de presse locale, dictionnaires biographiques (lesbiographies.com et Who’s Who in France), pages Wikipédia, professions de foi, présentations de soi sur les sites et réseaux sociaux des candidats, curriculum vitae disponibles en ligne, bases de données du ministère de l’Intérieur, et données de la HATVP
2. Constituer un jeu de données propre avec notamment les données de la HATVP ([lien](https://www.hatvp.fr/consulter-les-declarations/)) et d'autres données à réfléchir comme le RNE (https://www.data.gouv.fr/datasets/repertoire-national-des-elus-1/)

Dans tous les cas, prendre en compte les critiques sur ces bases de données et les catégories qu'ils ont mobilisés. Mais se souvenir aussi de ce qui nous intéresse ici (pas forcément les professions auto-déclarées mais les affiliations partisanes, et les contextes et manières d'utilliser la "République")

### Join deputy dataset with CRS dataset

In [ ]:
# Stabiliser le ID_orateur pour etre au format AN (et pouvoir matcher données)

import pandas as pd

df = pd.read_csv(
    "à compléter", low_memory=False, dtype={"ID_orateur": str} #à compléter = emplacement fichier par législature ou brut
)
df.shape

df["ID_orateur"] = "PA" + df["ID_orateur"]

In [ ]:
# Importer maintenant le fichier des données sur les députés à joindre
df_deputes = pd.read_csv(".../deputes-historique(datan-datagouv).csv")

In [ ]:
# shape avant fusion
df.shape

In [ ]:
# Join des données sur la base de l'id des intervenants
df = df.join(df_deputes.set_index("id"), on="ID_orateur", how="left", rsuffix="_dep")

In [ ]:
# shape après fusion
df.shape

In [ ]:
# certaines col du join introduisent une erreur à l'import/export
# forcer le QUOTE_ALL permet de résoudre

import csv  # pour résoudre le soucis d'écart. Checker

df.to_csv(
    "../data/interim/data_cleaning.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [ ]:
# verif excriture/lecture ok
df_test = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)
df_test.shape

## Étape 3 : fusionner les deux fichiers CSV 

### Attention aux différences : 

La présence en métadonnées de références "Sessionref" et "Séanceref" n'apparait qu'à la XVIe législature mais absent dans la 15e. Ne pose pas de soucis dans la mesure ou c'est présent sous une forme écrite "Session"/"Séance" pour les 2. 
*Pose la question de si pertinent de garder "Sessionref" et "Séanceref" ou si on les nettoie ?*

Absence aussi de Stime...mais pareil pas important. 

In [1]:
import pandas as pd

# Étape 1 : lire les deux fichiers CSV
fichier1 = '/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_15e.csv'
fichier2 = '/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_16e.csv'

df1 = pd.read_csv(fichier1)
df2 = pd.read_csv(fichier2)

# Étape 2 : fusionner les deux fichiers
df_fusion = pd.concat([df1, df2], ignore_index=True)

# Étape 3 : supprimer les doublons
    # On peut préciser des colonnes clés si on veut plus de précision
df_fusion = df_fusion.drop_duplicates()

# Étape 4 : (optionnelle) trier les données par date ou autre champ pertinent
if 'dateSeance' in df_fusion.columns:
    df_fusion['dateSeance'] = pd.to_datetime(df_fusion['dateSeance'], errors='coerce')
    df_fusion = df_fusion.sort_values(by='dateSeance')

# Étape 5 : sauvegarder le résultat
df_fusion.to_csv('/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS.csv', index=False, encoding='utf-8')

print("Fusion terminée : fichier sauvegardé sous 'Data_AN_CSS.csv'")


Fusion terminée : fichier sauvegardé sous 'Data_AN_CSS.csv'


### Checker les différences entre les 2 fichiers optenus (fusion de tous les xml d'un coup ou des 2 CSV après-coup)

Différents test (difflib echec) et les 2 fichiers sembles similaires en tout point ! Dans tous les cas, il existe une version séparée avec exactement la même extraction pour chaque législature au cas où pour appliquer des analyses spécifiques à chaque législature.

In [ ]:
import pandas as pd

df = pd.read_csv(
    "/Users/matthiaslevalet/Desktop/Projet de recherche/CSS_République/Data/Interim/Data_AN_CSS.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(1128128, 27)

In [6]:
df.groupby("session", dropna=False)["UID"].describe()

,count,unique,top,freq
session,,,,
Congrès du Parlement du 3 juillet 2017,86,2,CRSANR5L15S2017O1N001,43
Congrès du Parlement du 4 mars 2024,77,1,CRSCGR5L16S2024O1N001,77
Congrès du Parlement du 9 juillet 2018,127,1,CRSJOCGR5L15S2018E1N001,127
Deuxième session extraordinaire 2017,5240,8,CRSANR5L15S2017E2N008,940
Deuxième session extraordinaire 2018,12560,23,CRSANR5L15S2018E2N008,1058
Deuxième session extraordinaire 2019,8582,18,CRSANR5L15S2019E2N017,756
Deuxième session extraordinaire 2020,9468,18,CRSANR5L15S2020E2N011,919
Deuxième session extraordinaire 2021,650,1,CRSANR5L15S2021E2N001,650
Deuxième session extraordinaire 2023,7961,12,CRSANR5L16S2023E2N011,993
